# Odkrivanje enačb, 2. del: uporaba predznanja

## 1. Verjetnostne gramatike in knjižnica ProGED
1.1 Za odkrivanje enačb z verjetnostnimi gramatikami bomo uporabili knjižnico ProGED: https://github.com/brencej/ProGED.
Instaliramo jo z ukazom "pip install git+https://github.com/brencej/ProGED". Če se pojavijo težave, ustvari sveže Pythonovo okolje in poskusi instalirati tja.

Poglej si enostaven primer uporabe spodaj. Lahko se poigraš z različnimi nastavitvami in možnostmi. Celoten nabor modelov si lahko ogledaš v "ED.models". 

In [ ]:
import numpy as np
import pandas as pd

import ProGED as pg

from vajeED_2_podatki import *

In [ ]:
podatki = generiraj_newton(100)

np.random.seed(1)
ED = pg.EqDisco(data=podatki, 
                lhs_vars=["F"],
                rhs_vars=["m", "a"],
                sample_size=10)

ED.generate_models()
ED.fit_models()
ED.get_results()

1.2 Prostor možnih enačb, ki ga želimo preiskovati, definiramo z verjetnostno kontekstno-neodvisno gramatiko. ProGED-u jo podamo v obliki stringa, zapisanega po pravilih knjižnice NLTK:

- produkcijska pravila so oblike "A -> B1 ... Bn [p]", kjer je A neterminal, B1 ... Bn so terminali ali neterminali, p pa je verjetnost pravila
- verjetnosti pravil z istim neterminalom na levi strani se morajo sešteti v 1
- krajši zapis pravil z istim neterminalom na levi strani je "A -> B1 ... Bn [p] | D1 ... Dn [q] | ... " 
- terminalni simboli se označijo z enojnimi narekovaji, npr. "A -> B 'x'"
- v ProGED-u je terminalni simbol "C" poseben - označuje numerične konstante, katerih vrednosti iščemo z optimizacijo

Gramatika za linearne izraze dveh spremenljivk je

E -> E + V | V  
V -> 'x' | 'y'

Uporabi to gramatiko za odkrivanje enačbe $y = x_1 + 3x_2$. Podatke lahko generiraš z "generiraj_linearno". Koraki:
1. zapiši linearno gramatiko v formalizmu NLTK (ne pozabi da so tudi aritmetični operatorji terminalni simboli)
2. naredi ProGED-ov objekt GeneratorGrammar, ki kot argument sprejme string iz točke 1
3. gramatiko iz točke 2 podaj ProGED-ovemu EqDisco kot argument generator
4. generiraj 15 naključnih izrazov z ED.generate_models in si jih oglej
5. kliči še fit\_models in get\_results ter si oglej rezultate

Zdaj linearni gramatiki dodaj še numerično konstanto. Kako se spremenijo generirani modeli in rezultati?

1.3. Napiši gramatiko, ki generira polinome, in jo uporabi za odkrivanje energijskega zakona. Verjetno bo potrebno zgenerirati nekaj sto modelov. Če bo imel ProGED težave z generiranjem toliko unikatnih izrazov, mu lahko daš več poskusov za generiranje z nastavitvijo "strategy_settings = {"max_repeat":1000}." 

Na prejšnjih vajah smo videli, da je šum velik trn v peti odkrivanja enačb. Kako vpliva šum na odkrivanje enačb z verjetnostnimi gramatikami?

In [ ]:
podatki = generiraj_energijski_zakon(500, sum=0)



1.4. Loti se še naslednjih enačb:
1. projekcija na y os pri kroženju $y = r\sin(\omega t)$, kjer je konstanta (*generiraj_krozenje*).  
2. Lorenzov popravek $\gamma = \sqrt{1 - v^2/c^2}$, kjer je c hitrost svetlobe, približno $3\cdot10^8 m/s$ (*generiraj_lorenz*)

Za vsako od enačb napiši gramatiko, ki jo lahko odkrije. Premisli tudi, kakšne verjetnosti produkcij bi bile smiselne. Poskusi čim bolj omejiti prostor enačb, pri čemer naj ima gramatika še vedno vsaj eno rekurzivno pravilo. Gramatike lahko preizkusiš tudi na podatkih.


In [ ]:
gramatika1 = #DOPOLNI

gramatika1 = pg.GeneratorGrammar(gramatika1)
podatki = generiraj_krozenje(100)
ED = pg.EqDisco(data=podatki, 
                sample_size=50,
                lhs_vars=["y"],
                rhs_vars=["r", "t"],
                generator = gramatika1)
ED.generate_models()

In [ ]:
gramatika2 = # DOPOLNI

gramatika2 = pg.GeneratorGrammar(gramatika2)
podatki = generiraj_krozenje(100)
ED = pg.EqDisco(data=podatki, 
                sample_size=50,
                lhs_vars=["gamma"],
                rhs_vars=["v"],
                generator = gramatika2)
ED.generate_models()

## 2. Preštevanje dreves in verjetnosti

Za linearno gramatiko iz 1.2 ter vsako od gramatik iz 1.4. nariši:
1. graf števila dreves z dano višino
2. graf verjetnosti vseh dreves do vključno dane višine

Če želiš, lahko sam implementiraš rekurzivne enačbe s predavanj. Alternativno lahko uporabiš ProGED-ovi metodi "grammar.count_trees" in "grammar.count_coverage". Obema je treba podati začetni neterminalni simbol, ki mora biti objekt nltk.Nonterminal.



In [ ]:
from nltk import Nonterminal
import matplotlib.pyplot as plt



## 3. Merske enote

Upoštevanje predznanja o merskih enotah spremenljivk nam pogosto lahko močno zmanjša prostor enačb. 
ProGED nam to omogoča preko formalizma atributnih gramatik, ki jih pretvorimo v navadne konteksno-neodvisne gramatike. Za pogoste tipe gramatik ProGED že implementira njihove dimenzijsko-pravilne različice.
Enote predstavimo z vektorji v izbrani bazi osnovnih enot (recimo SI). 

3.1 Poglej si spodnji primer za Newtonov zakon ter dopolni manjkajoče!

Plonkec iz osnovne šole:
- Newtonov zakon $ F=m a$
- F je sila, z enoto kg*m/s^2
- m je masa, z enoto kg
- a je pospešek, z enoto m/s^2

In [ ]:
from ProGED.generators.dimensions import construct_dimensional_polynomial_grammar

podatki = generiraj_newton(100)

spremenljivke = ["m", "a"]
simboli_enot = ["m", "s", "kg"]
enote_na_desni = # DOPOLNI
enota_na_levi = # DOPOLNI

grammar = construct_dimensional_polynomial_grammar(variables = spremenljivke,
                                        units = enote_na_desni,
                                        target_variable_unit = enota_na_levi,
                                        unit_symbols = simboli_enot)

grammar

Vidimo, da ima gramatika veliko več simbolov in produkcij, kot tiste, ki smo jih pisali na roko. Zdaj uporabimo EqDisco in zgenerirajmo množico enačb.

In [ ]:
ED = pg.EqDisco(data=podatki, 
                lhs_vars=["F"],
                rhs_vars=["m", "a"],
                sample_size=10,
                generator = grammar)

ED.generate_models()


3.2. Poglejmo še pod krov, torej atributno gramatiko, ki izraža pravila za računanje z merskimi enotami. Oglej si spodnjo atributno gramatiko za dimenzijsko-pravilne polinome ter premisli, kako bi napisal dimenzijsko-pravilno atributno verzijo gramatik, ki si jih sestavil v nalogi 1.4. 

In [ ]:
from ProGED.generators.dimensions import dimensional_attribute_grammar_to_pcfg
from nltk import PCFG

prods = []
prods += [["P -> P '+' M", 0.4, ["uP1 - uP2", "uP1 - uM1"]]]
prods += [["P -> M", 0.6, ["uP1 - uM1"]]]
prods += [["M -> M '*' V", 0.4, ["-uM1 + uM2 + uV1"]]]
prods += [["M -> V", 0.4, ["uM1 - uV1"]]]
prods += [["M -> 'C'", 0.2, ["uM1"]]]

pcfg_start, pcfg_prods = dimensional_attribute_grammar_to_pcfg(prods, 
                                                               spremenljivke, 
                                                               enote_na_desni, 
                                                               enota_na_levi, 
                                                               append_vars = True, 
                                                               unit_symbols = simboli_enot)
grammar = pg.GeneratorGrammar(PCFG(pcfg_start, pcfg_prods))
grammar